In [13]:
# Todo: balance dataset?

# Sentiment Analysis

In [146]:
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# nltk.download('wordnet')
# from nltk.stem import WordNetLemmatizer
# from nltk.tokenize import word_tokenize



from keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding
from keras.datasets import imdb
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [147]:
data = pd.read_csv('emotions.csv')

display(
    data,
    print(data['label'].unique())
)

['anger' 'sadness' 'joy' 'love' 'fear' 'surprise']


,text,label
0,i feel so pissed off over an old friend and so...,anger
1,ive found it has made a huge difference especi...,anger
2,i also feel it is unfortunate that nearly all ...,sadness
3,i feel petty a href http clairee,anger
4,i used to believe that a feeling like fear was...,sadness
...,...,...
19995,i was i might be buying stuff from there but i...,joy
19996,i like sonam deepika and genelia who i feel ar...,joy
19997,i feel pathetic that i can hardly go a whole d...,sadness
19998,id have spent more time with her on reading i ...,sadness


None

In [148]:
# Find the longest "review" (for lack of a better word) and count the number of chars/words. Useful in order
# to set an appropriate max_words for pad_sequences.
longest_row_index = data['text'].str.len().idxmax()
longest_row = data['text'].loc[longest_row_index]

longest_review_char_count = len(longest_row)
longest_review_word_count = len(longest_row.split(' '))

print(f'Longest "review" characters/word count: {longest_review_char_count}/{longest_review_word_count}')

Longest "review" characters/word count: 300/62


In [156]:
# Tokenize the text data 
tokenizer = Tokenizer(num_words=1000) # Consider only using the top 1000 words, as those 

# "Learn" our dataset
tokenizer.fit_on_texts(data['text'])

sequences = tokenizer.texts_to_sequences(data['text'])

# printing using a loop for easier viewing
for i in range(5):
    print(sequences[i])

[1, 2, 14, 513, 161, 115, 76, 274, 277, 3, 68, 178]
[73, 322, 12, 99, 131, 6, 275, 29, 5, 24, 10, 3, 5, 10, 714, 119, 14, 84, 3, 152, 495]
[1, 116, 2, 12, 21, 762, 8, 35, 5, 964, 9, 87, 4, 782, 5, 367, 57, 27, 50]
[1, 2, 665, 6, 203, 187]
[1, 323, 4, 300, 8, 6, 7, 13, 763, 18, 4, 27, 614, 34, 111, 162, 37, 29, 22, 11, 6, 242]


In [ ]:
# Pad sequences to make them of uniform length
maxlen = 100  # Set the maximum length of sequences
data_pad = pad_sequences(sequences, maxlen=maxlen)

In [ ]:


# Create label encoding for the emotions
labels = pd.get_dummies(data['label'])

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(data_pad, labels, test_size=0.2, random_state=42)

# Define the RNN model
model = Sequential()
model.add(Embedding(max_words, 32, input_length=maxlen))  # Embedding layer
model.add(SimpleRNN(32))  # Simple RNN layer
model.add(Dense(4, activation='softmax'))  # Dense layer with softmax activation for multi-label classification

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
